In [472]:
%%capture
%run 02_account.ipynb

In [473]:
from objsize import get_deep_size as get_size

## Block

In [499]:
class Block: 
    def __init__(self, txs, prev_hash, number):
        self.txs       = []
        self.txs       = self.val_txs(txs)
        self.prev_hash = prev_hash
        self.number    = number
        self.time      = time.time()
        self.mined     = False
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        super().__setattr__('hash', sha(self.__dict__))
        
    def add(self, tx, is_coinbase=False):
        if not self.val_tx(tx): return False
        if is_coinbase: self.txs.insert(0, tx)
        else          : self.txs.append(tx)
        return True
            
    def is_dup(self, tx):
        tx_hashes = [tx.hash for tx in self.txs]
        return tx.hash in tx_hashes
    
    def val_tx(self, tx):
        assert val_sig(tx),         'tx signature invalid'
        assert not self.is_dup(tx), 'tx already in block'
        return True
    
    def val_txs(self, txs):
        for i,tx in enumerate(txs): 
            if not self.add(tx): txs.pop(i)
        return txs
        
    def __str__(self): 
        block_str = ('time:\t\t'        + time.ctime(self.time)+
                     '\nnumber:\t\t'    + str(self.number)+
                     '\nprev_hash:\t'   + ph(self.prev_hash)+
                     '\n\ntxs:\t\t\n\n' + txs2str(self.txs))
        if self.mined: 
            block_str += ('\nminer:\t\t' +ph(self.miner)+
                          '\nhash:\t\t'  +ph(self.hash)+
                          '\ndiff:\t\t'  +str(self.diff)+
                          '\nreward:\t\t'+str(self.reward)+
                          '\nnonce:\t\t' +str(self.nonce)) 
        return block_str
    def __bytes__(self): return (txs2str(self.txs)+
                                 self.prev_hash+
                                 str(self.number)+
                                 str(self.time)).encode()
    def __eq__(self, other): return self.hash == other.hash

In [500]:
acc1,acc2 = Account(),Account()

In [501]:
tx1 = acc1.signed_tx(acc2, 24, 0.1)
tx2 = acc2.signed_tx(acc1, 18, 0.12)

block = Block([tx1,tx2], rh(), 2); print(block)

time:		Sat Mar 27 12:39:29 2021
number:		2
prev_hash:	👹 131cabaf1a254f68...15f

txs:		

time:	Sat Mar 27 12:39:29 2021
from:	📝 0x7714D5E1Be38aA...9Fc
to:	📃 0x5d956f6611F823...6F9
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	📸 923d8aa1e636f7a0...5f5
signed:	true

time:	Sat Mar 27 12:39:29 2021
from:	📃 0x5d956f6611F823...6F9
to:	📝 0x7714D5E1Be38aA...9Fc
value:	18 ether
fee:	0.12 ether
nonce:	0
hash:	💳 4dc1db2053c57b09...fdb
signed:	true



Blocks can only contain unique txs. If you try to add a tx to a block that is already inclued it should fail.

In [502]:
dup_detected = False
try:    block.add(tx1)
except: dup_detected = True
assert  dup_detected

If anything in the tx is changed, like increasing the value to send, the signature should become invalid and the block creation should fail.

In [503]:
tx2.value = 1800

val_failed = False
try:    Block([tx1,tx2], rh(), 2)
except: val_failed = True
assert  val_failed

## Mining

Mining is a crucial component on any blockchain relying on proof of work (abbr. pow). It is used to reach consensus in a network of anonymous decentralized nodes.

Pow uses computational ressources to calculate a specific hash. This can only be done by brute force. 

In [516]:
class Miner:
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, 0, self.acc.nonce))
    
    def mine(self, block, diff, reward, attempts=1000):
        mb = deepcopy(block)
        mb.add(self.coinbase(reward), is_coinbase=True)
        mb_b = bytes(mb)
        nonce = 0
        for i in range(attempts):
            candidate   = mb_b + str(nonce).encode()
            candidate_h = sha(candidate)
            if candidate_h[:diff] == '0'*diff: break
            nonce += 1
        mb.diff   = diff
        mb.reward = reward
        mb.nonce  = nonce
        mb.miner  = self.pub
        mb.mined  = True
        return mb
    
    def mine_genesis(self, txs, reward=100):
        block = Block(txs, '0x0', 0)
        return self.mine(block, 0, reward)

In [505]:
miner = Miner(acc1)
mb = miner.mine(block, 2, 100); print(mb)

time:		Sat Mar 27 12:39:29 2021
number:		2
prev_hash:	👹 131cabaf1a254f68...15f

txs:		

time:	Sat Mar 27 12:39:33 2021
from:	📝 0x7714D5E1Be38aA...9Fc
to:	📝 0x7714D5E1Be38aA...9Fc
value:	100 ether
fee:	0 ether
nonce:	1
hash:	💠 3a64ab7e56dce932...e18
signed:	true

time:	Sat Mar 27 12:39:29 2021
from:	📝 0x7714D5E1Be38aA...9Fc
to:	📃 0x5d956f6611F823...6F9
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	📸 923d8aa1e636f7a0...5f5
signed:	true

time:	Sat Mar 27 12:39:29 2021
from:	📃 0x5d956f6611F823...6F9
to:	📝 0x7714D5E1Be38aA...9Fc
value:	1800 ether
fee:	0.12 ether
nonce:	0
hash:	📨 820b252912b9ab29...3df
signed:	true

miner:		📝 0x7714D5E1Be38aA...9Fc
hash:		🕅 dfb68b5eb94980f5...ec3
diff:		2
reward:		100
nonce:		140


In [506]:
assert len(mb.txs) == 3

Like a tx, if anything in the mined block changes the hash changes as well. This way we can make sure that the mined block is immutable.

In [507]:
mb_changed_reward        = deepcopy(mb)
mb_changed_reward.reward = 500

assert not mb == mb_changed_reward

### Genesis Block

Is the first block in the blockchain. This is how it all begins.

In [510]:
gb = miner.mine_genesis([tx1]); print(gb)

time:		Sat Mar 27 12:40:00 2021
number:		0
prev_hash:	0x0

txs:		

time:	Sat Mar 27 12:40:00 2021
from:	📝 0x7714D5E1Be38aA...9Fc
to:	📝 0x7714D5E1Be38aA...9Fc
value:	100 ether
fee:	0 ether
nonce:	5
hash:	🔙 b3697c0928f9a719...561
signed:	true

time:	Sat Mar 27 12:39:29 2021
from:	📝 0x7714D5E1Be38aA...9Fc
to:	📃 0x5d956f6611F823...6F9
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	📸 923d8aa1e636f7a0...5f5
signed:	true

miner:		📝 0x7714D5E1Be38aA...9Fc
hash:		🕓 ed710ce9930f450e...55e
diff:		1
reward:		100
nonce:		23
